In [1]:
import pandas as pd
import numpy as np
from misc_utils import *

N = 1000  # global variable for number of trials being use to generate random variables

**Graph 1:**

Unblockable backdoor path. Frontdoor path cannot be blocked either: unable to use rule 1 or rule 2 ($y \not\perp z_1$, and there are no variables on which we can condition to make the two independent).

**Graph 2:**

Blocking the backdoor path:
$$
\begin{align*}
P(y | \hat{x}, z_1) &= \sum_{z_3}P(y | \hat{x}, z_1, z_3)P(z_3 | \hat{x}, z_1) && \text{(condition on $z_3$)}\\
    &= \sum_{z_3}P(y | \hat{x}, \hat{z_1}, z_3)P(z_3 | \hat{x}, \hat{z_1}) && \text{(rule 2 on $z_1$)}\\
    &= \sum_{z_3}P(y | \hat{z_1}, z_3)P(z_3 | \hat{z_1}) && \text{(rule 3 on $\hat{x}$ in both terms)}\\
    &= \sum_{z_3}P(y, z_3 | \hat{z_1}) && \text{(definition of conditional probability)}\\
    &= P(y | \hat{z_1}) && \text{(un-condition on $z_3$)}\\
    &= P(y) && \text{(rule 3 on $z_1$)}
\end{align*}
$$

Using the formula $P(y | \hat{x}, z_1) = \frac{P(y, z_1 | \hat{x})}{P(z_1 | \hat{x})}$, we evaluate the numerator and denominator separately:

$$
\begin{align*}
P(y, z_1 | \hat{x}) &= \sum_{z_3} P(y, z_1 | \hat{x}, z_3)P(z_3 | \hat{x}) && \text{(condition on $z_3$)}\\
    &= \sum_{z_3} P(y | \hat{x}, z_1, z_3)P(z_1 | \hat{x}, z_3)P(z_3) && \text{(product rule, rule 3 on $\hat{x}$)}\\
    &= \sum_{z_3} P(y | z_3)P(z_1 | x, z_3)P(z_3) && \text{(based on working above, rule 2 on $\hat{x}$)}\\[20pt]
P(z_1 | \hat{x}) &= \sum_{z_3} P(z_1 | \hat{x}, z_3)P(z_3 | \hat{x})\\
    &= \sum_{z_3} P(z_1 | x, z_3)P(z_3)\\
    &= \sum_{z_3} P(z_1, z_3 | x)\\
    &= P(z_1 | x)
\end{align*}
$$

We can demonstrate that these two results are equal. We start by generating random variables:

In [2]:
# Start with z3
z3_list = np.random.rand(N)
z3_list = np.vectorize(lambda x: 1 if x > 0.4 else 0)(z3_list)  # P(z3 = 1) = 0.6

# x and y
x = []
y = []

for z3 in z3_list:
    if z3 == 0:
        x.append(1 if np.random.rand() > 0.7 else 0)   #P(x = 1|z3 = 0) = 0.3
        y.append(1 if np.random.rand() > 0.25 else 0)  #P(y = 1|z3 = 0) = 0.75
    else:
        x.append(1 if np.random.rand() > 0.3 else 0)   #P(x = 1|z3 = 1) = 0.7
        y.append(1 if np.random.rand() > 0.55 else 0)  #P(y = 1|z3 = 1) = 0.45

x_list = np.array(x)
y_list = np.array(y)

# z2
z2 = []
for y in y_list:
    if y == 0:
        z2.append(1 if np.random.rand() > 0.5 else 0)  #P(z_2 = 1|y = 0) = 0.5
    else:
        z2.append(1 if np.random.rand() > 0.1 else 0)  #P(z_2 = 1|y = 1) = 0.9
z2_list = np.array(z2)

# z1
z1 = []
for prod in zip(x_list, z2_list):
    if prod == (0, 0):
        z1.append(1 if np.random.rand() > 0.3 else 0)   #P(z_1 = 1|x = 0, z2 = 0) = 0.7
    elif prod == (0, 1):
        z1.append(1 if np.random.rand() > 0.15 else 0)  #P(z_1 = 1|x = 0, z2 = 1) = 0.85
    elif prod == (1, 0):
        z1.append(1 if np.random.rand() > 0.7 else 0)   #P(z_1 = 1|x = 1, z2 = 0) = 0.3
    elif prod == (1, 1):
        z1.append(1 if np.random.rand() > 0.95 else 0)  #P(z_1 = 1|x = 1, z2 = 1) = 0.05
    else:
        raise ValueError(f"Invalid cartesian product: {prod}.")
z1_list = np.array(z1)

df = pd.DataFrame({
    "x": x_list,
    "y": y_list,
    "z1": z1_list,
    "z2": z2_list,
    "z3": z3_list
})

display(df)

,x,y,z1,z2,z3
0,0,1,0,0,0
1,1,0,0,1,0
2,1,0,1,0,1
3,1,1,0,1,0
4,1,1,0,1,1
...,...,...,...,...,...
995,0,0,0,1,1
996,1,0,0,1,1
997,1,1,0,1,1
998,0,1,1,1,0


We now proceed with the calculations. We will calculate $P(y = 1 | \hat{x}, z_1)$ for all combinations of $\hat{x}$ and $z_1$. Since this is equivlent to $P(y)$ these should, in principle, all be the same.

In [13]:
import time
# We start with P(y = 1) (the first solution):
p_y = cond_prob(df, {"y": 1})
print(f"P(y = 1) = {p_y} with N = {N}")

# We proceed with the second solution:
for (x, z1) in [(0, 0), (0, 1), (1, 0), (1, 1)]:
    denominator = cond_prob(df, {"z1": z1}, {"x": x})
    numerator = sum_over_var(df, ["z3"], [({"y": 1, "z3": -1}, None), ({"z1": z1}, {"x": x, "z3": z3})])

    print(f"P(y = 1, z1 = {z1} | do(x = {x})) / P(z1 = {z1} | do(x = {x})) = {numerator / denominator} with N = {N}")

# It works!

P(y = 1) = 0.577 with N = 1000
P(y = 1, z1 = 0 | do(x = 0)) / P(z1 = 0 | do(x = 0)) = 0.5990684931506849 with N = 1000
P(y = 1, z1 = 1 | do(x = 0)) / P(z1 = 1 | do(x = 0)) = 0.572782722513089 with N = 1000
P(y = 1, z1 = 0 | do(x = 1)) / P(z1 = 0 | do(x = 1)) = 0.5831343380795175 with N = 1000
P(y = 1, z1 = 1 | do(x = 1)) / P(z1 = 1 | do(x = 1)) = 0.5373368825543516 with N = 1000
